# RAG = Vector Embedding + LLM

[RAG](https://python.langchain.com/v0.1/docs/modules/data_connection/)는 Retrieval-Augmented Generation의 약자로 LLM 모델이 특정 문서에 근거하여 보다 **정확한 답변**을 할 수 있도록 하는 기법. 자체적으로 구축한 DB나 외부 DB에서 사용자 질의에 대해 답변할 수 있는 근거 문서를 찾고, 이를 질문과 함께 LLM에게 전달하여 답변하게 한다. 이렇게 하므로써 LLM 모델이 학습하지 않은 내용에 대해 응답할 수있으며, 환각 현상 (hallucination) 이 줄어든 답변을 생성할 수있다.

## RAG와 Fine tuning

- **Fine tuning**
    - 범용적인 대용량의 데이터로 학습된 사전 학습 모델(pre-trained model)에 모델이 처리하기 바라는 특정 도메인 데이터를 추로 학습시켜 맞춤형 모델로 업데이트 하는 방식이다.
    - **장점**
      - 도메인 최적화되어 높은 성능을 낼 수있다.
    - **단점**
      - 학습에 시간과 노력이 크게 든다. 
      - 실시간 정보가 반영되지 못한다. 그래서 새로운 데이터가 업데이트 되는 경우 재학습을 시켜야 한다.
- **RAG**
  - 장점
    - 최신정보를 반영하는 것이 쉽다. 

## RAG 작동 단계
1. 정보 저장
    1. **Load**: 원본 text 데이터를 읽어온다.
    2. **Split**: Load한 데이터를 작은 덩어리(chunk)로 나눈다.
    3. **Embedding**: text를 embedding vector로 변환한다.
    4. **Store**: Embedding된 chunk를 vector db에 저장한다.
   
![rag](figures/rag1.png)

2. 탐색, 질의, 생성
    1. **Retrieve**: 질문과 관련있는 vector db에서 검색한다.
    2. **Query**: 질문과 검색한 내용을 prompt로 만들어 LLM에 질의한다.
    3. **Generation**: LLM은 받은 query에 대한 답변을 생성한다.
   
- RAG 흐름
  
![Retrieve and Generation](figures/rag2.png)


## RAG 작동 단계
1. 정보 저장
    1. **Load**: 원본 text 데이터를 읽어온다.
    2. **Split**: Load한 데이터를 작은 덩어리(chunk)로 나눈다.
    3. **Embedding**: text를 embedding vector로 변환한다.
    4. **Store**: Embedding된 chunk를 vector db에 저장한다.
   
![rag](figures/rag1.png)

2. 탐색, 질의, 생성
    1. **Retrieve**: 질문과 관련있는 vector db에서 검색한다.
    2. **Query**: 질문과 검색한 내용을 prompt로 만들어 LLM에 질의한다.
    3. **Generation**: LLM은 받은 query에 대한 답변을 생성한다.
   
- RAG 흐름
  
![Retrieve and Generation](figures/rag2.png)


# Loader

![rag_load](figures/rag_load.png)

- 다양한 문서 형식(format)에 맞춰 읽어오는 다양한 Loader 들이 있다.
  - https://python.langchain.com/docs/how_to/#document-loaders
  - 3rd party library(ppt, github 등등 다양한 3rd party lib도 있음. ) 들과 연동해 다양한 형식의 문서를 읽어올 수 있다.
    - https://python.langchain.com/docs/integrations/document_loaders/
- **모든 Loader는 기본적으로 동일한 interface(사용법)로 호출할 수있다.**
- **반환타입**
  - **list[Document]**
  - Load 한 문서는 Document객체에 정보들을 넣는다. 여러 문서를 읽을 수 있기 대문에 list에 묶어서 반환한다.
    - **Document 속성**
      - page_content: 문서의 내용
      - metadata(option): 문서에 대한 메타데이터(정보)를 dict 형태로 저장한다. 
      - id(option): 문서의 고유 id
## 주요 Loader

### Text file
- TextLoader 이용

### PDF
- PyPDF, Pymupdf 등 다양한 3rd party library를 이용해 pdf 문서를 읽어온다.
  - https://python.langchain.com/docs/integrations/document_loaders/#pdfs
- 설치 패키지
  - `pip install pypdf -qU`
  - `pip install pymupdf -qU`

### CSV
- CSVLoader

### Web

- WebBaseLoader 이용
- https://python.langchain.com/docs/how_to/document_loader_web/

### UnstructuredLoader
- 다양한 비정형 문서들을 읽어 오는 Unstrctured 를 사용해, 다양한 형식의 문서들을 load 해 RAG, 모델 파인튜닝에 적용할 수있게 한다.
  - 지원 파일 형식: "csv", "doc", "docx", "epub", "image", "md", "msg", "odt", "org", "pdf", "ppt", "pptx", "rtf", "rst", "tsv", "xlsx"

- **다양한 형식의 파일로 부터 text를 로딩**해야 할 경우 유용하다. 
- Local에 library를 설치해서 사용하거나,  Unstructured 가 제공하는 API service를 사용할 수 있다.
  - https://docs.unstructured.io
- 텍스트 파일, PDF, 이미지, HTML, XML, ms-office(word, ppt), epub 등 다양한 비정형 데이터 파일을 처리할 수 있다.
  - 설치, 지원 문서: https://docs.unstructured.io/open-source/installation/full-installation
  - Langchain 문서: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file

> - UnstructuredLoader PDF Load 시 Document 분할 기준
>     -  문서의 구조와 콘텐츠를 기반으로 텍스트를 분할해 Document에 넣는다.
>     -  분할 기준
>        - 헤더(Header): 문서의 제목이나 섹션 제목 등
>        - 본문 텍스트(NarrativeText): 일반적인 문단이나 설명문
>        - 표(Table): 데이터가 표 형식으로 구성된 부분
>        - 리스트(List): 순서가 있거나 없는 목록
>        - 이미지(Image): 사진이나 그래픽 요소

#### 설치할 프로그램
- poppler
  - pdf 파일을 text로 변환하기 위해 필요한 프로그램
  - windows: https://github.com/oschwartz10612/poppler-windows/releases/ 에서 최신 버전 다운로드 후 압축 풀어서 설치.
    - 환경변수 Path에 "설치경로\Library\bin" 을 추가. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install poppler`
  - Linux: `sudo apt-get install poppler-utils`
- tesseract-ocr
  - OCR 라이브러리로 pdf 이미지를 text로 변환하기 위해 필요한 프로그램 
  - windows: https://github.com/UB-Mannheim/tesseract/wiki 에서 다운받아 설치. 
    - 환경변수 Path에 설치 경로("C:\Program Files\Tesseract-OCR") 추가 한다. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install tesseract`
  - linux(unbuntu): `sudo apt install tesseract-ocr`
- 설치 패키지
  - `pip install "unstructured[pdf]" -qU`
  - libmagic 설치
      - windows: `pip install python-magic-bin -qU`
      - macOS: `brew install libmagic`
      - linux(ubuntu): `sudo apt-get install libmagic-dev`
  - `pip install langchain-unstructured -qU`

### Directory 내의 문서파일들 로딩
- DirectoryLoader 이용